#Demo de 3 Agentes BDI interactuando para controlar la apertura de una puerta.
Los agentes son:
1. un agente PORTERO (o PORTER), que cierra y abre la puerta de la habitación si se solicita.
2. un agente PARANOICO (o PARANOID), que prefiere que la puerta esté cerrada con llave y le pide al PORTERO que cierre la puerta si así lo desea.
3. un agente CLAUSTROFÓBICO (o CLAUSTROPHOBE), que prefiere que la puerta esté abierta y le pide al PORTERO que abra la puerta.
La simulación ejecuta N iteraciones del escenario, y en cada iteración, cada agente actúa una vez.

Fuente: https://github.com/TimKam/JS-son/blob/master/examples/node/beliefPlan.js

Librería: https://github.com/TimKam/JS-son
       (basada en JavaScript pero adaptada para ejecutar en Colab)

#Entorno:

In [1]:
#@title Instalar librerías necesarias
!pip install --upgrade jinja2==3.0.2
!pip install pixiedust_node

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.8/133.8 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.4
    Uninstalling Jinja2-3.1.4:
      Successfully uninstalled Jinja2-3.1.4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.6/197.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pixiedust_node: filename=pixiedust_node-0.2.5-py2.py3-none-any.whl size=9386 sha256=e8fa44acf6dc68a88e3c486f9a08371ca40c13a900158ebce75476314188a18f
  Stored in directory: /root/.cache/pip/wheels/58/4e/c0/78dcb7f3755b97d2dc4ae2d714f97526fd266032ad52b96268
  Created wheel for pixiedust: filename=pixiedust-1.1.19-py3-none-any.whl size=321786 sha256=0bfed50ecd834bde7cf6d7135ae78f7c644cd4a82780dcf7665d2acc57a1f226
  Stored in directory: /root/.cache/pip/wheels/e7/

In [2]:
#@title Importar librería para ejecutar JS en Colab
#@markdown Nota: si lanza error volver a ejecutar y funciona ;-)
try:
  import pixiedust_node
except:
  import pixiedust_node

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [3]:
#@title Importar librería JS-son
npm.install('js-son-agent')

/tools/node/bin/npm install -s js-son-agent
pixiedust_node 0.2.5 started. Cells starting '%%node' may contain Node.js code.


In [4]:
#@title Define variables generales
## We import js-son and assign Belief, Plan, Agent, and Environment to sepearate consts for the sake of convenience
%%capture
%%node
const {
  Belief,
  Desire,
  Plan,
  Agent,
  Environment
} = require('js-son-agent')

#Agentes y Enviroment:

In [5]:
#@title Definir Beliefs generales
#All agents start with the same belief set.
#The belief with the ID ``door`` is assigned the object ``{ locked: true}``.I.e., the door is locked.
#Also, nobody has so far requested any change in door state (``requests: []``).
%%capture
%%node

const beliefs = {
  ...Belief('door', { locked: true }),
  ...Belief('requests', [])
}


In [6]:
#@title Definir agente PORTER
#The porter has the following plans:
## 1. If it does not believe the door is locked and it has received a request to lock the door (head), lock the door (body).
## 2. If it believes the door is locked and it has received a request to unlock the door (head),    unlock the door (body).
%%capture
%%node
const plansPorter = [
  Plan(
    beliefs => !beliefs.door.locked && beliefs.requests.includes('lock'),
    () => [{ door: 'lock' }]
  ),
  Plan(
    beliefs => beliefs.door.locked && beliefs.requests.includes('unlock'),
    () => [{ door: 'unlock' }]
  )
]

const porter = new Agent('porter', beliefs, {}, plansPorter)

In [7]:
#@title Definir agente PARANOID
#The paranoid agent has the following plans:
## 1. If it does not belief the door is locked (head), it requests the door to be locked (body).
## 2. If it beliefs the door is locked (head), it broadcasts a thank you message for locking the door (body).
%%capture
%%node
const plansParanoid = [
  Plan(
    beliefs => !beliefs.door.locked,
    () => [{ request: 'lock' }]
  ),
  Plan(
    beliefs => beliefs.door.locked,
    () => [{ announce: 'gracias por cerrar la puerta!' }]
  )
]

const paranoid = new Agent('paranoid', beliefs, {}, plansParanoid)


... ... ... ... ... ...


In [8]:
#@title Definir agente CLAUSTROPHOBE
#The claustrofovic agent has the following plans:
##1. If it beliefs the door the door is locked (head), it requests the door to be unlocked (body).
## 2. If it does not belief the door is locked (head),    it broadcasts a thank you message for unlocking the door (body).
%%capture
%%node
const plansClaustrophobe = [
  Plan(
    beliefs => beliefs.door.locked,
    () => [{ request: 'unlock' }]
  ),
  Plan(
    beliefs => !beliefs.door.locked,
    () => [{ announce: 'gracias por abrir la puerta!' }]
  )
]

const claustrophobe = new Agent('claustrophobe', beliefs, {}, plansClaustrophobe)


... ... ...


In [9]:
#@title Especificar el Enviroment
#First, we set the environments state, which is--in our case--consistent with the agents' beliefs:
#Next, To define how the environment processes agent actions, we implement the ``updateState`` function.
##The function takes an agent's actions, as well as the agent ID and the current state to determine
##the environment's state update that is merged into the new state
##``state = { ...state, ...stateUpdate }``.
#Then, to simulate a partially observable world, we can specify the environment's ``stateFilter`` function,
##which determines how the state update should be shared with the agents.
##However, in our case we simply communicate the whole state update to all agents,
##which is also the default behavior of the environment, if no ``stateFilter`` function is specified.
#Last, We instantiate the environment with the specified agents, state, update function, and filter function
%%capture
%%node

const state = {
  door: { locked: true },
  requests: []
}


const updateState = (actions, agentId, currentState) => {
  const stateUpdate = {
    requests: currentState.requests
  }
  actions.forEach(action => {
    if (action.some(action => action.door === 'lock')) {
      stateUpdate.door = { locked: true }
      stateUpdate.requests = []
      console.log(`${agentId}: Lock door`)
    }
    if (action.some(action => action.door === 'unlock')) {
      stateUpdate.door = { locked: false }
      stateUpdate.requests = []
      console.log(`${agentId}: Unlock door`)
    }
    if (action.some(action => action.request === 'lock')) {
      stateUpdate.requests.push('lock')
      console.log(`${agentId}: Request: lock door`)
    }
    if (action.some(action => action.request === 'unlock')) {
      stateUpdate.requests.push('unlock')
      console.log(`${agentId}: Request: unlock door`)
    }
    if (action.some(action => action.announce)) {
      console.log(`${agentId}: ${action.find(action => action.announce).announce}`)
    }
  })
  return stateUpdate
}

const stateFilter = state => state

const environment = new Environment(
  [paranoid, claustrophobe, porter],
  state,
  updateState,
  stateFilter
)

#Ejecución:

In [10]:
#@title Indicar cantidad de ciclos de Ejecución (N)
N = 10 #@param {"type":"integer"}
if N<5:
  N=5
import time
time.sleep(1) # para que se sincronice

... ... ... ... ... ... ... ... ...
... ... ... ... ... ... ... ... ...
... ... ... ... ... ... ... ... ...
... ... ...
... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ...
... ... ... ... ...


In [11]:
#@title Ejecutar el enviroment
%%capture
%%node
environment.run(N)

porter: Unlock door
paranoid: Request: lock door
claustrophobe: gracias por abrir la puerta!
porter: Lock door
paranoid: gracias por cerrar la puerta!
claustrophobe: Request: unlock door
porter: Unlock door
paranoid: Request: lock door
claustrophobe: gracias por abrir la puerta!
porter: Lock door
paranoid: gracias por cerrar la puerta!
claustrophobe: Request: unlock door
porter: Unlock door
paranoid: Request: lock door
claustrophobe: gracias por abrir la puerta!
porter: Lock door
paranoid: gracias por cerrar la puerta!
claustrophobe: Request: unlock door
